Animal classification Model

For training

Taking input from predicate-matrix continuous and storing it in numpy array

In [ ]:
# Function to read the file and convert to a 2D array
def read_2d_array_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    # Convert each line to a list of integers and store in a 2D array
    array_2d = [list(map(float, line.strip().split())) for line in lines]
    
    return array_2d

# Example usage
filename = 'predicate-matrix-continuous.txt'  # Predicate-matrix-continuous file path
array_2d = read_2d_array_from_file(filename)

In [2]:
import numpy as np
predicate_continuous = np.array(array_2d) # Convert to numpy array
predicate_continuous[predicate_continuous < 0] = 0 # Replace negative values with 0
del array_2d # Free up memory
# Visualize the 2D array
print(predicate_continuous.shape)
print(predicate_continuous)

(50, 85)
[[ 0.    0.    0.   ...  2.35  9.7   8.38]
 [39.25  1.39  0.   ... 58.64 20.14 11.39]
 [83.4  64.79  0.   ... 15.77 13.41 15.42]
 ...
 [63.57 43.1   0.   ... 35.95 28.26  5.  ]
 [55.31 55.46  0.   ...  5.04 18.89 72.99]
 [10.22 21.53 27.73 ...  3.96 14.05 37.98]]


Taking the data(names of animals) from classes.txt 

In [3]:
class_dir = 'classes.txt' # Path to the classes.txt file
classes = []
with open (class_dir, 'r') as f:
    for line in f:
        line = line.split('\n')[0]
        if line.strip():  # Ignore empty lines
            _, name = line.split(maxsplit=1)  # Split into number and name
            classes.append(name)
# Ensure data is loaded correctly
print(len(classes))
classes

50


['antelope',
 'grizzly+bear',
 'killer+whale',
 'beaver',
 'dalmatian',
 'persian+cat',
 'horse',
 'german+shepherd',
 'blue+whale',
 'siamese+cat',
 'skunk',
 'mole',
 'tiger',
 'hippopotamus',
 'leopard',
 'moose',
 'spider+monkey',
 'humpback+whale',
 'elephant',
 'gorilla',
 'ox',
 'fox',
 'sheep',
 'seal',
 'chimpanzee',
 'hamster',
 'squirrel',
 'rhinoceros',
 'rabbit',
 'bat',
 'giraffe',
 'wolf',
 'chihuahua',
 'rat',
 'weasel',
 'otter',
 'buffalo',
 'zebra',
 'giant+panda',
 'deer',
 'bobcat',
 'pig',
 'lion',
 'mouse',
 'polar+bear',
 'collie',
 'walrus',
 'raccoon',
 'cow',
 'dolphin']

Splitting the data into training and validation

In [4]:
import os
train_dir = "vlg-dataset/train" # Path to the train directory
images_path_train =[]
images_path_val = []
y_train = np.zeros((7635,85))
y_val = np.zeros((1909,85))
i,j,k,l = 0,0,0,0
for subfolder in classes:
    subfolder_path =os.path.join(train_dir, subfolder)
    if os.path.exists(subfolder_path):
        for filename in os.listdir(subfolder_path):
            img_path =os.path.join(subfolder_path, filename)
            #Putting every 5th image in the validation set
            if k%5==0:
                images_path_val.append(img_path)
                y_val[l] = predicate_continuous[i]
                l+=1
            else:
                images_path_train.append(img_path)
                y_train[j] = predicate_continuous[i]
                j += 1
            k+=1
    i += 1

# Ensure data is loaded correctly
print(len(images_path_val), len(images_path_train))
print(y_val.shape, y_train.shape)

1909 7635
(1909, 85) (7635, 85)


Converting into numpy arrays

In [5]:
x_train = np.array(images_path_train)
x_val = np.array(images_path_val)

del images_path_train, images_path_val # Free up memory
# Ensure data is loaded correctly
print(x_train.shape, x_val.shape)
print(y_train.shape, y_val.shape)


(7635,) (1909,)
(7635, 85) (1909, 85)


Loading the dataset

In [6]:
import tensorflow as tf

# Load and preprocess the images into 346x346x3 matrix
def preprocess_image(image_path, label, target_size=346):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, target_size, target_size)
    image = image / 255.0  # Normalize to [0, 1]
    #Rotate the image randomly
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    #Flip the image randomly
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    return image,label

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((x_train ,y_train))
dataset = dataset.map(preprocess_image , num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(32)

# Use this dataset for training
for images, label in dataset.take(1):  # Example batch
    print(images.shape)  # (batch_size, 346, 346, 3)
    print(label.shape)  # (batch_size, predicates_size)

# Create a TensorFlow dataset for validation
val_dataset = tf.data.Dataset.from_tensor_slices((x_val ,y_val))
val_dataset = val_dataset.map(preprocess_image , num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(16)

# Use this dataset for validation
for images, label in dataset.take(1):  # Example batch
    print(images.shape)  # (batch_size, 346, 346, 3)
    print(label.shape)  # (batch_size, predicates_size)

(32, 346, 346, 3)
(32, 85)
(32, 346, 346, 3)
(32, 85)


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics, losses

# Feature extractor (CNN backbone)
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', pooling='avg')
base_model.trainable = False
# Define the model
model = models.Sequential([
    layers.InputLayer(input_shape=(346,346,3)),
    base_model,
    layers.Dense(512, activation='relu'),
    layers.Dense(85)
])
# Compile the model
model.compile(
    optimizer = Adam(),
    loss = losses.CosineSimilarity(axis = -1),
    metrics = [metrics.CosineSimilarity(axis = -1)]
)
# Train the model
model.fit(
    dataset,
    validation_data = val_dataset,
    epochs = 30
)

Epoch 1/30
239/239 [==============================] - 62s 244ms/step - loss: -0.8790 - cosine_similarity: 0.8790 - val_loss: -0.8949 - val_cosine_similarity: 0.8949
Epoch 2/30
239/239 [==============================] - 57s 233ms/step - loss: -0.9269 - cosine_similarity: 0.9269 - val_loss: -0.9089 - val_cosine_similarity: 0.9089
Epoch 3/30
239/239 [==============================] - 56s 232ms/step - loss: -0.9374 - cosine_similarity: 0.9374 - val_loss: -0.9116 - val_cosine_similarity: 0.9116
Epoch 4/30
239/239 [==============================] - 57s 234ms/step - loss: -0.9461 - cosine_similarity: 0.9461 - val_loss: -0.9205 - val_cosine_similarity: 0.9205
Epoch 5/30
239/239 [==============================] - 57s 235ms/step - loss: -0.9521 - cosine_similarity: 0.9521 - val_loss: -0.9131 - val_cosine_similarity: 0.9131
Epoch 6/30
239/239 [==============================] - 56s 229ms/step - loss: -0.9549 - cosine_similarity: 0.9549 - val_loss: -0.9240 - val_cosine_similarity: 0.9240
Epoch 7/30

In [17]:
model.save('Animal_classification_model.h5') # Save the model

Trying with training the base model also

In [16]:
# Train the model
base_model.trainable = True
model.fit(
    dataset,
    validation_data=val_dataset,
    epochs = 25

)

Epoch 1/25
239/239 [==============================] - 54s 223ms/step - loss: -0.9743 - cosine_similarity: 0.9743 - val_loss: -0.9516 - val_cosine_similarity: 0.9516
Epoch 2/25
239/239 [==============================] - 54s 223ms/step - loss: -0.9742 - cosine_similarity: 0.9742 - val_loss: -0.9484 - val_cosine_similarity: 0.9484
Epoch 3/25
239/239 [==============================] - 54s 223ms/step - loss: -0.9734 - cosine_similarity: 0.9734 - val_loss: -0.9472 - val_cosine_similarity: 0.9472
Epoch 4/25
239/239 [==============================] - 54s 224ms/step - loss: -0.9765 - cosine_similarity: 0.9765 - val_loss: -0.9502 - val_cosine_similarity: 0.9502
Epoch 5/25
239/239 [==============================] - 54s 224ms/step - loss: -0.9757 - cosine_similarity: 0.9757 - val_loss: -0.9530 - val_cosine_similarity: 0.9530
Epoch 6/25
239/239 [==============================] - 54s 222ms/step - loss: -0.9759 - cosine_similarity: 0.9759 - val_loss: -0.9487 - val_cosine_similarity: 0.9487
Epoch 7/25

In [ ]:
model.save('Animal_classification_model(2).h5') # Save the model

For testing

Loading the model

In [15]:
from tensorflow.keras.models import load_model
model = load_model('Animal_classification_model(2).h5', compile = False)

Taking the index of seen classes and unseen classes

In [9]:
import os
train_dir = 'vlg-dataset/train'
i=0
unseen_classes_index = []
seen_classes_index = []
for label in classes:
    if label in os.listdir(train_dir):
        seen_classes_index.append(i)
    else:
        unseen_classes_index.append(i)
    i+=1   

print(unseen_classes_index)
print(seen_classes_index)

[6, 15, 19, 21, 22, 24, 26, 27, 28, 45]
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 23, 25, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49]


Defining the function for classification of an image

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf

def preprocess_image_2(image_path, target_size=346):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, target_size, target_size)
    image = image / 255.0  # Normalize to [0, 1]

    return tf.expand_dims(image,axis=0) # expanding the dimension in order to fit the batch size of 1

#Using 2 types of evaluation for image classification
def evaluate_image(image_path):
    image = preprocess_image_2(image_path)
    embedding_pred = model.predict(image)
    similiarity = cosine_similarity(embedding_pred, predicate_continuous)
    predicted_label_index = tf.argmax(similiarity, axis =1).numpy()[0]
    print(classes[predicted_label_index])
    return classes[predicted_label_index]

#using this so that the model does not get biased towards seen classes
def evaluate_image_2(image_path):
    threshold_perc= 0.89
    image = preprocess_image_2(image_path)
    embedding_pred = model.predict(image)
    similarity = cosine_similarity(embedding_pred, predicate_continuous)
    for i in seen_classes_index:
        similarity[0,i] *= threshold_perc
    predicted_label_index = tf.argmax(similarity, axis=1).numpy()[0]
    print(classes[predicted_label_index])
    return classes[predicted_label_index]

Classify a single image

In [ ]:
image_path = 'vlg-dataset/test/00010.jpg'
print(evaluate_image_2(image_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
lion
lion


Making classification of multiple images and storing them into a csv file

In [16]:
import os
import csv

# Path to the test directory
test_dir = 'vlg-dataset/test'

# Define the output CSV file path
output_csv = "predictions2.csv"

# Ensure the output CSV file is initialized with headers
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image_id", "class"])  # Writing the header row

# Process images and directly append results to the CSV file
for filename in os.listdir(test_dir):
    image_path = os.path.join(test_dir, filename)

    # Evaluate the image
    pred = evaluate_image_2(image_path)

    # Write the prediction result directly to the file
    with open(output_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([filename, pred])  # Append the prediction

print(f"Predictions saved directly to '{output_csv}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
mouse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
humpback+whale
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
mouse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
spider+monkey
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
cow
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
persian+cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
mouse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
cow
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
cow
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
lion
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
raccoon
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
cow
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
mouse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
persian+cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
mouse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
humpback+whale
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
rat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
mouse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
cow
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
mouse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
tiger
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213